# Model validation with domamin metrics

In [ ]:
# Read shapefiles

pred_boxes = gpd.read_file('/content/drive/MyDrive/NOVA_course_deep_learning/data/tiles/full_mod.shp')

ref_boxes = gpd.read_file('/content/drive/MyDrive/NOVA_course_deep_learning/data/map_data/test_annotations2_sun.geojson')
aois = gpd.read_file('/content/drive/MyDrive/NOVA_course_deep_learning/data/map_data/aois.geojson')  # Shapefile containing multiple AOIs


### 7.1 Installing required packages and mounting Google Drive

In [ ]:
!pip install geopandas
import geopandas as gpd
import numpy as np

In [ ]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

### 7.2 Plots

In [ ]:
import matplotlib.pyplot as plt

# Plotting the shapefiles

fig, ax = plt.subplots(figsize=(10, 10))

# Plot predicted boxes

pred_boxes.plot(ax=ax, color='red', edgecolor='black', alpha=0.5, label='Predicted Boxes')

# Plot reference boxes

ref_boxes.plot(ax=ax, color='blue', edgecolor='black', alpha=0.5, label='Reference Boxes')

# Plot AOI

aois.plot(ax=ax, color='green', edgecolor='black', alpha=0.5, label='AOI')

# Zoom to the extent of predicted boxes

ax.set_xlim(pred_boxes.total_bounds[0], pred_boxes.total_bounds[2])
ax.set_ylim(pred_boxes.total_bounds[1], pred_boxes.total_bounds[3])

# Set plot title and legend

plt.title('Tree Bounding Boxes')
plt.legend()

# Display the plot

plt.show()

### 7.3 Compute residuals

In [ ]:
# Initialize lists to store residuals for each area of interest (AOI)

residuals = []

# Compute residuals for each AOI

for aoi in aois.iterrows():
    aoi_geom = aoi[1].geometry
    pred_within_aoi = pred_boxes[pred_boxes.geometry.within(aoi_geom)]
    ref_within_aoi = ref_boxes[ref_boxes.geometry.within(aoi_geom)]

    num_pred_within_aoi = len(pred_within_aoi)
    num_ref_within_aoi = len(ref_within_aoi)

    residual = num_pred_within_aoi - num_ref_within_aoi
    residuals.append(residual)


### 7.4 Compute RMSE

In [ ]:
# Compute RMSE

rmse = np.sqrt(np.mean(np.power(residuals, 2)))
bias= np.mean(residuals)
print("the RMSE is: "+ str(rmse))
print("the bias is: "+ str(bias))

In [ ]:
# Area of interest

aois

In [ ]:
# Print residuals and RMSE

for i, aoi in enumerate(aois.iterrows()):
    aoi_name = aoi[1]['aoi_name']       # Assuming you have a column 'aoi_name' in your AOIs shapefile
    print("Residual for AOI"+ str(aoi_name) + ":" +str(residuals[i]))

print("RMSE:"+  str(rmse))

### 7.5 Calculating per hectare values

In [ ]:
# Initialize lists to store the number of boxes per hectare for each AOI

box_density = []

# Compute the number of boxes per hectare for each AOI

for aoi in aois.iterrows():
    aoi_geom = aoi[1].geometry
    pred_within_aoi = pred_boxes[pred_boxes.geometry.within(aoi_geom)]
    ref_within_aoi = ref_boxes[ref_boxes.geometry.within(aoi_geom)]

    area = aoi_geom.area        # Area in square meters
    area_ha = area / 10000      # Convert area to hectares
    num_pred_within_aoi = len(pred_within_aoi)
    num_ref_within_aoi = len(ref_within_aoi)

    pred_density = num_pred_within_aoi / area_ha
    ref_density = num_ref_within_aoi / area_ha

    box_density.append((pred_density, ref_density))

In [ ]:
box_density

In [ ]:
# Print box density for each AOI

for i, aoi in enumerate(aois.iterrows()):
    aoi_name = aoi[1]['aoi_name']       # Assuming you have a column 'aoi_name' in your AOIs shapefile
    pred_density, ref_density = box_density[i]
    print("Box density for AOI"+ str(aoi_name) + ":")
    print("Predicted density (boxes/ha):"+ str(pred_density))
    print("Reference density (boxes/ha):"+ str(ref_density))
    print()


In [ ]:
# Compute RMSE of box density

pred_densities, ref_densities = zip(*box_density)
residuals = np.array(pred_densities) - np.array(ref_densities)
rmse = np.sqrt(np.mean(np.power(residuals, 2)))
bias = np.mean(residuals)

rmse_rel= rmse/np.mean(ref_densities)*100
bias_rel= bias/np.mean(ref_densities)*100

print("RMSE of box density (boxes/ha):"+ str(rmse))
print("relative RMSE of box density (%):"+ str(rmse_rel))

print("bias of box density (boxes/ha):"+ str(bias))
print("relative bias of box density (%):"+ str(bias_rel))
